# Analisis de prestaciones por diagnostico

- En este apartado sequiere ver la cantidad de prestaciones que se realizan por cada diagnostico principal.

- El resultado que se quiere obtener es parecido al siguiente:

|Año|Diagnostico|Resumen Procedimientos|
|---|-----------|----------------------|
|2019|"I25.1"|{"45.23": 20, "73.59": 10}|
|2020|"I25.1"|{"45.23": 7, "73.59": 3}|